In [66]:
from lib.data.DataPreProcessor import DataPreProcessor
from lib.db.repositories.kreise_sterberate_repo import KreiseSterberateRepository
from lib.config.Configuration import Configuration
from lib.db.connection_factory import ConnectionFactory
from sklearn.model_selection import train_test_split

ConnectionFactory.add_connection(Configuration.db_connection)

In [67]:
pre_processor = DataPreProcessor()
ks_repo = KreiseSterberateRepository('kreise_sterberate', Configuration.db_connection)

In [68]:
df = ks_repo.find_all()
df

,index,Landkreis,ID_LK_SK,LK_SK,Bundesland,age_covid_0_34_%,age_covid_35_59_%,age_covid_60+_%,age_covid_unknown_%,gender_covid_f_%,gender_covid_m_%,gender_covid_unknown_%,Sterberate_%
0,0,LK Ahrweiler,7131,LK,Rheinland-Pfalz,39.5480,42.3729,16.9492,1.129940,46.7949,50.6410,2.564100,0.564972
1,1,LK Aichach-Friedberg,9771,LK,Bayern,18.9474,44.5614,36.4912,0.000000,51.3410,48.2759,0.383142,7.017540
2,2,LK Alb-Donau-Kreis,8425,LK,Baden-Württemberg,25.0804,48.8746,26.0450,0.000000,55.6797,44.3203,0.000000,3.376210
3,3,LK Altenburger Land,16077,LK,Thüringen,15.8730,52.3810,31.7460,0.000000,54.8387,45.1613,0.000000,4.761900
4,4,LK Altenkirchen,7132,LK,Rheinland-Pfalz,22.2222,39.5062,38.2716,0.000000,61.7450,38.2550,0.000000,6.790120
...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,396,SK Worms,7319,SK,Rheinland-Pfalz,35.1220,43.9024,20.4878,0.487805,50.2538,49.7462,0.000000,3.414630
397,397,SK Wuppertal,5124,SK,Nordrhein-Westfalen,26.5711,37.2657,36.1632,0.000000,56.0274,43.8356,0.136986,8.930540
398,398,SK Würzburg,9663,SK,Bayern,29.2473,27.3118,43.2258,0.215054,53.2319,46.7681,0.000000,10.537600
399,399,SK Zweibrücken,7320,SK,Rheinland-Pfalz,36.1111,41.6667,22.2222,0.000000,58.3333,41.6667,0.000000,2.777780


In [69]:
features_raw, targets_raw = pre_processor.split_features(df.drop(['index', 'Landkreis', 'Bundesland', 'ID_LK_SK', 'LK_SK'], axis=1), ['Sterberate_%'])
features_raw

,age_covid_0_34_%,age_covid_35_59_%,age_covid_60+_%,age_covid_unknown_%,gender_covid_f_%,gender_covid_m_%,gender_covid_unknown_%
0,39.5480,42.3729,16.9492,1.129940,46.7949,50.6410,2.564100
1,18.9474,44.5614,36.4912,0.000000,51.3410,48.2759,0.383142
2,25.0804,48.8746,26.0450,0.000000,55.6797,44.3203,0.000000
3,15.8730,52.3810,31.7460,0.000000,54.8387,45.1613,0.000000
4,22.2222,39.5062,38.2716,0.000000,61.7450,38.2550,0.000000
...,...,...,...,...,...,...,...
396,35.1220,43.9024,20.4878,0.487805,50.2538,49.7462,0.000000
397,26.5711,37.2657,36.1632,0.000000,56.0274,43.8356,0.136986
398,29.2473,27.3118,43.2258,0.215054,53.2319,46.7681,0.000000
399,36.1111,41.6667,22.2222,0.000000,58.3333,41.6667,0.000000


In [70]:
features_final = pre_processor.one_hot_encode(features_raw)
list(features_final.columns)

['age_covid_0_34_%',
 'age_covid_35_59_%',
 'age_covid_60+_%',
 'age_covid_unknown_%',
 'gender_covid_f_%',
 'gender_covid_m_%',
 'gender_covid_unknown_%']

In [71]:
# Split the 'features' and 'income' data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_final, 
                                                    targets_raw, 
                                                    test_size = 0.2, 
                                                    random_state = 0)

# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 320 samples.
Testing set has 81 samples.


In [72]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier

learner1 = LinearRegression()
learner1.fit(X_train, y_train)
learner1.score(X_test, y_test)

0.6038719247259243